In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import os
import copy
from tqdm import tqdm

In [13]:
# Enable CUDNN for performance optimization
torch.backends.cudnn.benchmark = True

In [23]:
# Define dataset paths
DATASET_PATH = "../FBMM/Unsplitted_Ready_Sets/set_01_class_balanced_augs_applied_splitted"
TRAIN_DIR = os.path.join(DATASET_PATH, "train")
VAL_DIR = os.path.join(DATASET_PATH, "val")
TEST_DIR = os.path.join(DATASET_PATH, "test")

# Define emotion classes
emotion_classes = ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]
num_classes = len(emotion_classes)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
# Image transformations (Resize & Normalize, No Augmentation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet-B0 expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

In [25]:
# Load datasets
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=transform)
val_dataset = datasets.ImageFolder(VAL_DIR, transform=transform)
test_dataset = datasets.ImageFolder(TEST_DIR, transform=transform)

In [26]:
# Create DataLoaders with optimized memory management
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [27]:
# Load EfficientNet-B0 model
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

In [28]:
# Freeze initial layers (Unlock only last layer + 20% of other final layers)
total_layers = len(list(model.features.children()))
unfreeze_from = int(total_layers * 0.8)

for i, child in enumerate(model.features.children()):
    if i < unfreeze_from:
        for param in child.parameters():
            param.requires_grad = False  # Freeze 80% of the layers
    else:
        for param in child.parameters():
            param.requires_grad = True  # Train 20% of layers

In [29]:
# Modify classifier for 7 emotion classes
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_ftrs, num_classes)
)

In [10]:
# Move model to device
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Early stopping variables
early_stop_patience = 5
best_val_loss = float("inf")
epochs_no_improve = 0

# Training Loop
num_epochs = 50
best_model_wts = copy.deepcopy(model.state_dict())

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 50)

    # Training Phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_progress = tqdm(train_loader, desc=f"Training {epoch+1}/{num_epochs}", leave=False)
    for images, labels in train_progress:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        # Forward
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        loss.backward()
        optimizer.step()

        # Metrics
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Update tqdm progress bar
        train_progress.set_postfix(loss=loss.item())

    train_loss = running_loss / len(train_dataset)
    train_acc = 100 * correct / total
    print(f"✅ Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")

    # Validation Phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    val_progress = tqdm(val_loader, desc=f"Validation {epoch+1}/{num_epochs}", leave=False)
    with torch.no_grad():
        for images, labels in val_progress:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            val_progress.set_postfix(loss=loss.item())

    val_loss = val_loss / len(val_dataset)
    val_acc = 100 * correct / total
    print(f"🟢 Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.2f}%")

    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), "best_emotion_model.pth")
        print("✅ Best model saved!")
        epochs_no_improve = 0  # Reset early stopping counter
    else:
        epochs_no_improve += 1

    # Check early stopping
    if epochs_no_improve >= early_stop_patience:
        print("⏳ Early stopping triggered!")
        break

# Load best model weights
model.load_state_dict(best_model_wts)

# Save final trained model
torch.save(model.state_dict(), "final_emotion_model.pth")
print("🎯 Final model saved!")


Epoch 1/50
--------------------------------------------------


In [11]:
# Training Loop
num_epochs = 50
best_model_wts = copy.deepcopy(model.state_dict())

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 50)

    # Training Phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        loss.backward()
        optimizer.step()

        # Metrics
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_dataset)
    train_acc = 100 * correct / total
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")

    # Validation Phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_loss = val_loss / len(val_dataset)
    val_acc = 100 * correct / total
    print(f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.2f}%")

    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), "best_emotion_model.pth")
        print("✅ Best model saved!")
        epochs_no_improve = 0  # Reset early stopping counter
    else:
        epochs_no_improve += 1

    # Check early stopping
    if epochs_no_improve >= early_stop_patience:
        print("⏳ Early stopping triggered!")
        break

# Load best model weights
model.load_state_dict(best_model_wts)

# Save final trained model
torch.save(model.state_dict(), "final_emotion_model.pth")
print("🎯 Final model saved!")


Epoch 1/50
--------------------------------------------------


KeyboardInterrupt: 